Creating SQL Databases

In [27]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("LearnSpark") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()


In [9]:
# spark.sql('DROP DATABASE learn_spark_db');

In [28]:
spark.sql("Create database learn_spark_db")
spark.sql("use learn_spark_db") # this command makes me go inside the database

24/08/17 20:27:42 WARN ObjectStore: Failed to get database learn_spark_db, returning NoSuchObjectException
24/08/17 20:27:42 WARN ObjectStore: Failed to get database learn_spark_db, returning NoSuchObjectException
24/08/17 20:27:42 WARN ObjectStore: Failed to get database learn_spark_db, returning NoSuchObjectException


DataFrame[]

In [ ]:
# spark.conf.set("spark.sql.legacy.createHiveTableByDefault", "false")


Creating Managed Table

In [12]:
spark.sql("""
    CREATE TABLE learn_spark_db.managed_us_delay_flights_table (
        date STRING,
        delay STRING,
        distance INT,
        origin STRING,
        destination STRING
    ) USING parquet
""")


DataFrame[]

getting the data to be saved in this table

In [17]:
flight_df = spark.read.csv("/Users/saurabhverma/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv", header= True, inferSchema= True)

In [18]:
flight_df.show(10)

+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
+-------+-----+--------+------+-----------+
only showing top 10 rows



In [19]:
flight_df.write.mode("append").saveAsTable("managed_us_delay_flights_table")

24/08/16 16:11:44 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [21]:
spark.sql("""CREATE TABLE us_delay_flights_tbl(date STRING, delay INT, distance INT, origin STRING, destination STRING) USING csv OPTIONS (PATH '/Users/saurabhverma/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/departuredelays.csv')""")

AnalysisException: [TABLE_OR_VIEW_ALREADY_EXISTS] Cannot create table or view `spark_catalog`.`learn_spark_db`.`us_delay_flights_tbl` because it already exists.
Choose a different name, drop or replace the existing object, or add the IF NOT EXISTS clause to tolerate pre-existing objects.

In [23]:
spark.sql("SHOW TABLES").show()

+--------------+--------------------+-----------+
|     namespace|           tableName|isTemporary|
+--------------+--------------------+-----------+
|learn_spark_db|managed_us_delay_...|      false|
|learn_spark_db|us_delay_flights_tbl|      false|
+--------------+--------------------+-----------+



In [24]:
spark.catalog.listDatabases()

[Database(name='default', catalog='spark_catalog', description='default database', locationUri='file:/Users/saurabhverma/Spark-Training/Chapter%25204/spark-warehouse'),
 Database(name='learn_spark_db', catalog='spark_catalog', description='', locationUri='file:/Users/saurabhverma/Spark-Training/Chapter%25204/spark-warehouse/learn_spark_db.db')]

In [25]:
spark.catalog.listTables()

[Table(name='managed_us_delay_flights_table', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='MANAGED', isTemporary=False),
 Table(name='us_delay_flights_tbl', catalog='spark_catalog', namespace=['learn_spark_db'], description=None, tableType='EXTERNAL', isTemporary=False)]

In [26]:
spark.catalog.listColumns('us_delay_flights_tbl')

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

In [27]:
spark.catalog.listColumns('managed_us_delay_flights_table')

[Column(name='date', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='delay', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='distance', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False),
 Column(name='origin', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='destination', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False)]

24/08/16 22:24:25 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1965419 ms exceeds timeout 120000 ms
24/08/16 22:24:25 WARN SparkContext: Killing executors is not supported by current scheduler.
24/08/16 22:24:28 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at 

Parquet

In [28]:
#reading parquet file in dataframe

In [9]:
file_loc = '/Users/saurabhverma/Spark-Training/Chapter 3/fire_sf_data/part-00001-0d37f043-a69e-4c15-a42f-5c2297aa8074-c000.snappy.parquet'

all_file_loc = '/Users/saurabhverma/Spark-Training/Chapter 3/fire_sf_data/*'

df = spark.read.format('parquet').load(file_loc)


In [8]:
df.show(10)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|      Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+--

In [15]:
#reading parquet file in spark sql table directly

# Register the Parquet file as a temporary view
spark.sql("CREATE OR REPLACE TEMPORARY VIEW my_parquet_view USING parquet OPTIONS (path '/Users/saurabhverma/Spark-Training/Chapter 3/fire_sf_data/' )")

# Query the data using Spark SQL
result_df = spark.sql("SELECT * FROM my_parquet_view ")

# Show the results
result_df.show()


# in sql if we want to create a spark unmanaged file then we can directly run the command


# CREATE OR REPLACE TEMPORARY VIEW my_parquet_view USING parquet OPTIONS (path '/Users/saurabhverma/Spark-Training/Chapter 3/fire_sf_data/' )

# an unmanaged file is something that is not stored within spark

+----------+------+--------------+-----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+--------------+---------+
|CallNumber|UnitID|IncidentNumber|         CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|         RowID|    Delay|
+----------+------+--------------+-----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------

In [16]:
#reading parquet file in spark sql table after getting the data in df

df.createOrReplaceTempView("my_parquet_view")

# Query the data using Spark SQL
result_df = spark.sql("SELECT * FROM my_parquet_view ")

# Show the results
result_df.show()


+----------+------+--------------+--------------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|            CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+--------------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

In [19]:
#writing dataframes to a spark sql tables

flight_df.write .mode("overwrite").saveAsTable('fligt_df_as_sql_table')

24/08/17 15:51:35 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/08/17 15:51:35 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/08/17 15:51:36 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/08/17 15:51:36 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore saurabhverma@127.0.0.1
24/08/17 15:51:36 WARN ObjectStore: Failed to get database default, returning NoSuchObjectException
24/08/17 15:51:37 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
24/08/17 15:51:38 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/08/17 15:51:38 WARN HiveConf: HiveConf of name hive.internal.s

In [20]:
#reading json into a df

json_file_path = '/Users/saurabhverma/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/summary-data/json/*'

json_df = spark.read.format('json').load(json_file_path)

In [21]:
json_df.show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [ ]:
#json can be read similarly into a sql table. first make the view and then read the view. spark view does not really save the data but points to the location

# Storage:
# SQL View: Does not store data; stores a query definition.
# Spark Temporary View: Does not store data separately; data is held in the DataFrame or Dataset that the view references.

In [24]:
spark.sql("CREATE OR REPLACE TEMPORARY VIEW us_delay_flights_tbl USING json OPTIONS ( path '/Users/saurabhverma/LearningSparkV2/databricks-datasets/learning-spark-v2/flights/summary-data/json/*' )")

spark.sql("SELECT * FROM us_delay_flights_tbl").show(10)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
|    United States|          Singapore|    1|
|    United States|            Grenada|   62|
|       Costa Rica|      United States|  588|
|          Senegal|      United States|   40|
|          Moldova|      United States|    1|
+-----------------+-------------------+-----+
only showing top 10 rows



In [26]:
#write dataframe to json in a similar manner as writing to parquet
#reading and writing csv is similar
#same for avro